In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (8955 Collins Ave, Surfside, FL 33154, USA, (2...
2     (125 Ocean Dr, Miami Beach, FL 33139, USA, (25...
3     (19122 Fisher Island Dr, Miami Beach, FL 33109...
4     (21050 NE 38th Ave APT 3004, Aventura, FL 3318...
5     (618 Valencia Ave #101, Coral Gables, FL 33134...
6     (9 Island Ave, Miami Beach, FL 33139, USA, (25...
7     (6000 Collins Ave APT 529, Miami Beach, FL 331...
8     (7000 Island Blvd APT 1607, Aventura, FL 33160...
9     (400 Sunny Isles Blvd, Sunny Isles Beach, FL 3...
10    (3131 NE 7th Ave #2005, Miami, FL 33137, USA, ...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Arte Surfside\r8955 Collins Ave LPH\rSurfside,12/22/2022,43,"$18,000,000.00",8027,"$3,552.40",Craig Heger,Echo Fine Properties,Oren Alexander,Official,Arte Surfside 8955 Collins Ave LPH Surfside,Arte Surfside,8955 Collins Ave LPH Surfside,"(8955 Collins Ave, Surfside, FL 33154, USA, (2...","(25.8758779, -80.1211034, 0.0)",25.875878,-80.121103,0.0
2,Ocean House\r125 Ocean Dr PH3\rMiami Beach,12/22/2022,1,"$9,200,000.00",2303,"$3,994.79",Eloy Carmenate,The Corcoran Group,Taylor Richardson,The Corcoran Group,Ocean House 125 Ocean Dr PH3 Miami Beach,Ocean House,125 Ocean Dr PH3 Miami Beach,"(125 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7700519, -80.132576, 0.0)",25.770052,-80.132576,0.0
3,Seaside\r19122 Fisher Island Dr 19122\rMiami B...,12/21/2022,156,"$3,000,000.00",2049,"$1,464.13",Robert Vole,The Island,Tatyana Ionin,Coldwell Banker Realty,Seaside 19122 Fisher Island Dr 19122 Miami Beach,Seaside,19122 Fisher Island Dr 19122 Miami Beach,"(19122 Fisher Island Dr, Miami Beach, FL 33109...","(25.7577288, -80.1386049, 0.0)",25.757729,-80.138605,0.0
4,Atlantic III\r21050 NE 38th Ave 3004\rAventura,12/20/2022,242,"$1,625,000.00",2970,$547.14,Stuart Berger,Coldwell Banker Realty,Emma Hazzan,Elite International Realty Inc,Atlantic III 21050 NE 38th Ave 3004 Aventura,Atlantic III,21050 NE 38th Ave 3004 Aventura,"(21050 NE 38th Ave APT 3004, Aventura, FL 3318...","(25.9714195, -80.1250866, 0.0)",25.971419,-80.125087,0.0
5,The New French Village\r618 Valencia Ave 101\r...,12/19/2022,4,"$1,315,000.00",2010,$654.23,James Byrd,Coldwell Banker Realty,David Armas,Biscayne Capital Real Estate,The New French Village 618 Valencia Ave 101 Co...,The New French Village,618 Valencia Ave 101 Coral Gables,"(618 Valencia Ave #101, Coral Gables, FL 33134...","(25.7469076, -80.2671671, 0.0)",25.746908,-80.267167,0.0
6,9 Island\r9 Island Ave 408\rMiami Beach,12/20/2022,61,"$1,200,000.00",1837,$653.24,Juan Carlos Gonzalez PA,Keller Williams Miami Beach Realty,Juan Carlos Gonzalez PA,Keller Williams Miami Beach Realty,9 Island 9 Island Ave 408 Miami Beach,,9 Island 9 Island Ave 408 Miami Beach,"(9 Island Ave, Miami Beach, FL 33139, USA, (25...","(25.790337, -80.14877249999999, 0.0)",25.790337,-80.148772,0.0
7,Terra Beachside\r6000 Collins Ave 529\rMiami B...,12/20/2022,136,"$1,200,000.00",2049,$585.65,Adriana Spielkamp,"AG Real Estate Advisors, LLC",Deborah Jerussalmy Corch,AMG International Realty,Terra Beachside 6000 Collins Ave 529 Miami Beach,Terra Beachside,6000 Collins Ave 529 Miami Beach,"(6000 Collins Ave APT 529, Miami Beach, FL 331...","(25.8429318, -80.1211127, 0.0)",25.842932,-80.121113,0.0
8,Williams Island\r7000 Island Blvd 1607\rAventura,12/19/2022,220,"$1,100,000.00",3180,$345.91,Mary Cases,"Cases Realty Group, Inc.",Joel Matus,"Williams Island Realty, LLC",Williams Island 7000 Island Blvd 1607 Aventura,Williams Island,7000 Island Blvd 1607 Aventura,"(7000 Island Blvd APT 1607, Aventura, FL 33160...","(25.9443918, -80.1356453, 0.0)",25.944392,-80.135645,0.0
9,400 Sunny Isles\r400 Sunny Isles Blvd 704\rSun...,12/22/2022,49,"$1,080,000.00",1705,$633.43,Sandra Silva,Fortune Int'l Realty Aventura Inc,Cinzia Francesca Iacoponi Escobar,Keller Williams Elite Properties,400 Sunny Isles 400 Sunny Isles Blvd 704 Sunny...,,400 Sunny Isles 400 Sunny Isles Blvd 704 Sunny...,"(400 Sunny Isles Blvd, Sunny Isles Beach, FL 3...","(25.9291821, -80.12832829999999, 0.0)",25.929182,-80.128328,0.0
10,0One Paraiso\r3131 NE 7th Ave 2005\rMiami,12/20/2022,43,"$1,045,000.00",1088,$960.48,Adam Redolfi,Barnes Int'l Realty LLC,Sandra Pariente,Charles Rutenberg Realty Fort,0One Paraiso 3131 NE 7th Ave 2005 Miami,,0One Paraiso 3131 NE 7th Ave 2005 Miami,"(3131 NE 7th Ave #2005, Miami, FL 33137, USA, ...","(25.8072459, -80.18560169999999, 0.0)",25.807246,-80.185602,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[6,'building_name']=('9 Island')
df.at[6,'address_only']=('9 Island Ave 408')

df.at[9,'building_name']=('400 Sunny Isles')
df.at[9,'address_only']=('400 Sunny Isles Blvd 704')

df.at[10,'building_name']=('One Paraiso')
df.at[10,'address_only']=('3131 NE 7th Ave 2005')

In [11]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('December 18th - December 24th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [15]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_122422
